# Python MySQL
- CRUD Operation

In [1]:
import json
import pymysql

In [2]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config = json.loads(config_str)

In [3]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port'] 
)

- 테이블 생성

In [4]:
sql_user = """
    CREATE TABLE if NOT EXISTS users (
        uid VARCHAR(20) NOT NULL PRIMARY KEY,
        pwd CHAR(44),
        uname VARCHAR(20) NOT NULL,
        reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        is_deleted INT DEFAULT 0
    );
"""

In [5]:
cur = conn.cursor()
cur.execute(sql_user)

0

- 테이블 변경

In [6]:
cur.execute('alter table users add column email varchar(40);')

0

- 데이터 입력

In [7]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid, uname) VALUES('admin','관리자');")

1

In [8]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 49, 8), 0, None)


In [9]:
# 입력한 데이터를 즉시 flush 하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid, uname) 
        VALUES('eskim','김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()

In [11]:
sql_select = """
    SELECT uid, uname, 
    DATE_FORMAT(reg_date, '%Y-%m-%d %h:%i') AS reg_date,
    is_deleted, email FROM users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-06-09 10:49', 0, None)
('eskim', '김은숙', '2021-06-09 10:55', 0, None)
('wjlee', '이우정', '2021-06-09 10:55', 0, None)


In [14]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

In [15]:
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid,))
conn.commit()

In [16]:
users = (('jbpark','박재범'),('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
# 아래 for 문은 cur.executemany(sql, users) 로 대체 가능
for user in users:
    cur.execute(sql, user)
conn.commit()

- 데이터 삭제

In [17]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin';"
cur.execute(sql)
conn.commit()

In [18]:
import pandas as pd
uids = ['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids, 'uname':unames})
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [19]:
cur = conn.cursor()
sql = "INSERT INTO users(uid, uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

- 데이터 수정

In [20]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
cur.execute(sql, (email, uid))
conn.commit()